
# ASDC SSWP Colour Picker Notebook

Preparatory steps and workflow for Colour Picker functionality of the ASDC/SSWP API 

Developed by 2Pi Software

Dane Evans

## Function of the SSWP Colour Picker

The SSWP Colour Picker examines sets of images to identify pixels with colours falling in a specified range. To assist interaction of determining the colour range, a sample colour 'Swatch' is referenced.

The Swatch may be a small cropped area from a target image, a composite of selected target pixel spreads, or a generated colour gradient.

When the Colour Picker is run, positive image results will be returned with bounding boxes marked. Bounding box extents as image co-ordinates are also output as text.

Colour ranges can be assessed in either RGB or HSV colour spaces. For the swatch to determine the range, it is first subject to clustering, establishing a majority colour spread, then the calculated spread is bracketed. (Defaulting to two bands of clustering as foreground vs background and retaining 98% spread of the majority band.)

Colour Picking and range extraction from Swatches can be chained into a single automated run or can be run as distinct processes.

In [ ]:
# --- Initialisation ----

# Imports 
import requests 
import json
import time
from getpass import getpass
import ipywidgets as widgets

# For dev, allow refresh of support lib
import importlib

import ASDC_API_support
importlib.reload(ASDC_API_support)
print("Lib loaded")

# Useful links to remember
site_url = "https://dev2pi.sswp.cosinecrm.com.au"
base_url = site_url + "/sswpapps-api/"
auth_url = base_url + "auth"

# Establish API authorisation

## Setup your ASDC user access

1. go to https://dev2pi.sswp.cosinecrm.com.au
2. log in to your SSWP account 
3. go to SSWP Apps > Manage account
4. set an App password using the Update Password interface in the middle of the screen



## Obtain a token for API access
1. go to https://dev2pi.sswp.cosinecrm.com.au/sswpapps-api/auth
2. log in with your ASDC user name and password
3. copy out the token, and paste it into the following cell as a string
4. (revisit the link to refresh or revoke your token at any time)


# Use a token to access API endpoints



In [ ]:
#@title
# Initialise the API 

token = getpass("Paste your ASDC API token here:")     
asdc_api = ASDC_API_support.ASDC_API_wrappers(token)

## The Info endpoint describes all available actions.

In [ ]:
asdc_api.format_human_readable(asdc_api.info_request())

## Example: The Uploads endpoint summarises your stored files.

In [ ]:
uploads = asdc_api.asdc_uploads()
try:
    uploads = uploads.json()['response']['payload']['uploads']
    list_uploads = []
    for i in uploads:
      list_uploads.append(str(i.get('id')) + ": " + str(i.get('title')))
except KeyError:
    print("No uploads.")

upload_picker = widgets.Dropdown(options=list_uploads)
upload_picker

# Workflows



## To make image sets available
- Nominate images for upload using 'public facing' URI's
- Upload sets of images including colour range swatches
- Keep track of your uploads by title and id


In [ ]:
## Enter URI's, whitespace separated or on multiple lines
#  Enter a blank last line to finish
uploading_uri_set = []
while True:
  uri_block = input()
  if uri_block == "":
    break
  uri_block = uri_block.strip()
  upload_additions =  uri_block.split()
  [uploading_uri_set.append(y) for y in (upload_additions)]

print("Confirming upload URI's:")
print(uploading_uri_set)

In [ ]:
## Start the upload process 
process_title = input("Enter a unique upload title: ")

process = asdc_api.asdc_processes_source(process_title, uploading_uri_set)

if process.status_code != 500:
  process_ids = []
  for i in process.json()['response']['payload']['launched']:
    process_ids.append(int(i.get('id')))
  print(process_ids)

  # Poll for the processes to finish 
  for i in range(30):
    print("Polling: " + str(i))
    done = True
    for process in process_ids:
      # explicitly note the process type as "source" for status of upload
      refresh = asdc_api.asdc_process_refresh(str(process)+"/source")
      message = refresh.json()['response']['payload']['process'][0]['message']
      print("Process " + str(process) + ": " + message)
      done &= ("Uploaded" in message)
    if done: 
      print("Processes completed. ")
      i=0
      break
    time.sleep(1)
  if (i==29):
    print("Processing is slow, please check SSWP console.")
else: # 500 error
  print(process.reason)
  print(process.json()['response']['payload'])

## To create colour ranges from swatches
- List your uploads
- Select a swatch set upload by title and id
- Start a "range" process 
- Poll until it process completes
- Get back colour range files

In [ ]:
## Get a list of your uploads 

uploads = asdc_api.asdc_uploads()
#asdc_api.format_human_readable(uploads)
uploads = uploads.json()['response']['payload']['uploads']
list_uploads = []
for i in uploads:
  list_uploads.append(str(i.get('id')) + ": " + str(i.get('title')))

print("Please select an upload:")
upload_picker = widgets.Dropdown(options=list_uploads)
upload_picker

In [ ]:
## After selecting an upload above, select an image to be a swatch 
selected_upload = upload_picker.value.split(':')[0]

# get the file list from the selected upload
upload_details = asdc_api.asdc_uploads(selected_upload, get_data=True)
#asdc_api.format_human_readable(upload_details)
files = upload_details.json()['response']['payload']['uploads'][0]['results']
list_files = []
for i in files:
  list_files.append(str(i.get('file')) + ": " + str(i.get('filename')))

print("Please select a swatch image:")
swatch_picker = widgets.Dropdown(options=list_files)
swatch_picker

In [ ]:
## Start the looped processes
process_title = input("Enter a unique processing title: ") ## BandSpreadOnBoneSeed

selected_upload = upload_picker.value.split(':')[0];
selected_swatches = {swatch_picker.value.split(':')[0]} ##, 3153, 3154, 3155, 3156}
run_for_bands = {1,2,3}
run_for_spread = {25,72,88,98}
colorspace = "BGR"

process_ids = []

for sw in selected_swatches:
  for bd in run_for_bands:
    for sp in run_for_spread:

      process = asdc_api.asdc_processes_range(str(process_title)+":"+str(sw)+":"+str(bd)+":"+str(sp), sw, selected_upload, colorspace="BGR", bands=bd, spread=sp)
      if process.status_code != 500:
        for i in process.json()['response']['payload']['launched']:
          process_ids.append(int(i.get('id')))

print(process_ids)

# Poll for the processes to finish 
for i in range(30):
  print("Polling: " + str(i))
  done = True
  for j in process_ids:
    refresh = asdc_api.asdc_process_refresh(j)
    message = refresh.json()['response']['payload']['process'][0]['message']
    print("Process " + str(j) + ": " + message)
    done &= message == "Results available"
  if done: 
    print("Processes completed. ")
    i=0
    break
  time.sleep(1)
if (i==29):
    print("Processing is slow, please check SSWP console.")
else: # 500 error
    print(process.reason)
    print(process.json()['response']['payload'])

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def get_color_gradient(c1, c2, n):
    """
    Adapted from sketch by kaggle.com/code/brendanartley
    """
    assert n > 1
    c1_rgb = np.array(c1)/255
    c2_rgb = np.array(c2)/255
    mix_pcts = [x/(n-1) for x in range(n)]
    rgb_colors = [((1-mix)*c1_rgb + (mix*c2_rgb)) for mix in mix_pcts]
    return ["#" + "".join([format(int(round(val*255)), "02x") for val in item]) for item in rgb_colors]


pieces = 120
min_id = min(process_ids)
max_id = max(process_ids)

## view results data 
for process in [*range(min_id, max_id, 1)] :
  apiGave = asdc_api.asdc_swatches(process, get_data=True)
  asdc_api.format_human_readable(apiGave)
  ranged = apiGave.json()['response']['payload']['swatches'][0]['results']['range']
  lbound = (ranged['Low'].replace('\n','').replace('{','').replace('}','').split())
  ubound = (ranged['High'].replace('\n','').replace('{','').replace('}','').split())

  lbound = [int(asStr) for asStr in lbound[0:3]]
  lbound[0],lbound[2] = lbound[2],lbound[0]
  ubound = [int(asStr) for asStr in ubound[0:3]]
  ubound[0],ubound[2] = ubound[2],ubound[0]

  plt.figure(figsize=(4,4))
  plt.pie([2 for i in range(pieces*6)], colors = get_color_gradient(lbound, ubound, pieces))
  plt.axis('equal')
  plt.title("Gradient View")
  plt.show()

## To run colour picking against an upload
- List your uploads
- Select an upload by title and id
- Start a "picker" process 
- Poll until it process completes
- Get back matches with bounding boxes

In [ ]:
## Get a list of your uploads 

uploads = asdc_api.asdc_uploads()
#asdc_api.format_human_readable(uploads)
uploads = uploads.json()['response']['payload']['uploads']
list_uploads = []
for i in uploads:
  list_uploads.append(str(i.get('id')) + ": " + str(i.get('title')))

print("Please select an upload:")
selected_picker = widgets.Dropdown(options=list_uploads)
selected_picker

In [ ]:
## Start the chained processes
process_title = input("Enter a unique processing title: ")

selected_swatch = 324
selected_picker = selected_picker.value.split(':')[0]

# Adapt parameters if you wish
# colorspace = "BGR"
# bands = 2
# spread = 82

process = asdc_api.asdc_processes_picker(process_title, selected_swatch, selected_picker)
#print(process.json())

if process.status_code != 500:
    process_ids = []
    for i in process.json()['response']['payload']['launched']:
        process_ids.append(int(i.get('id')))
        print(process_ids)

    # Poll for the processes to finish 
    for i in range(30):
        print("Polling: " + str(i))
        done = True
        for j in process_ids:
            refresh = asdc_api.asdc_process_refresh(j)
            message = refresh.json()['response']['payload']['process'][0]['message']
            print("Process " + str(j) + ": " + message)
            done &= message == "Results available"
        if done: 
            print("Processes completed. ")
            i=0
            break
        time.sleep(1)
        if (i==29):
            print("Processing is slow, please check SSWP console.")
        else: # 500 error
            print(process.reason)
            print(process.json()['response']['payload'])

In [ ]:
## view results data 
for process in process_ids:
  print(process)
  asdc_api.format_human_readable(asdc_api.asdc_runs(process, get_data=True))

In [ ]:
## Chase links from results per above
result_link_details = asdc_api.asdc_runs(process_ids[0], get_data=True)
files = result_link_details.json()['response']['payload']['runs'][0]['results']

list_files = []
for i in files:
  list_files.append(str(i.get('get_link')))
print("Please select a results download request:")
link_picker = widgets.Dropdown(options=list_files)
link_picker

In [ ]:
## Get a directly accessible link:
print(asdc_api.asdc_download_by_link(request_url=link_picker.value).content)

In [ ]:
# ## Start the chained processes
# process_title = input("Enter a unique processing title: ")

# selected_swatch = swatch_picker.value.split(':')[0]

# # Adapt parameters if you wish
# colorspace = "BGR"
# bands = 2
# spread = 82

# process = asdc_api.asdc_processes_chain(process_title, selected_swatch, selected_upload, colorspace, bands, spread)
# #print(process.json())

# if process.status_code != 500:
#   process_ids = []
#   for i in process.json()['response']['payload']['launched']:
#     process_ids.append(int(i.get('id')))
#   print(process_ids)

#   # Poll for the processes to finish 
#   for i in range(30):
#     print("Polling: " + str(i))
#     done = True
#     for process in process_ids:
#       refresh = asdc_api.asdc_process_refresh(process)
#       message = refresh.json()['response']['payload']['process'][0]['message']
#       print("Process " + str(process) + ": " + message)
#       done &= message == "Results available"
#     if done: 
#       print("Processes completed. ")
#       i=0
#       break
#     time.sleep(1)
#   if (i==29):
#     print("Processing is slow, please check SSWP console.")
# else: # 500 error
#   print(process.reason)
#   print(process.json()['response']['payload'])

# Custom Workflow

In [ ]:
import matplotlib.pyplot as plt
import ASDC_API_helper

#### Authenticate with API token

In [ ]:
helper = ASDC_API_helper.Helper()

### **Uploads**

#### Select from a list of my uploaded image sets

In [ ]:
helper.select_upload()

#### Get a sample of the files in my selected image set

In [ ]:
print("Files in this upload:")
for key in [x for x in helper.selection['upload']][:5]:
    print(key, helper.selection['upload'][key]['filename'])
if len(helper.selection['upload']) > 5:
    print("...")

#### Get the image content for each image in the set

In [ ]:
helper.selection['upload'] = helper.get_image_content(helper.selection['upload_id'], helper.selection['upload'])

#### Plot all images in the set

In [ ]:
helper.plot_figures(helper.selection['upload'], ncols=5)

### **Runs**

#### Select from a list of my runs

In [ ]:
helper.select_run()

#### Get a sample of the files in my selected image set

In [ ]:
print("Files in this run:")
for key in [x for x in helper.selection['run']]:
    print(key, helper.selection['run'][key]['filename'])
# if len(helper.selection['run']) > 5:
#     print("...")

#### Get the image content for each image in the set

In [ ]:
helper.get_image_arrays(helper.selection['run_root_id'], helper.selection['run'])

#### Plot all images in the set

In [ ]:
helper.plot_figures(helper.selection['run'], ncols=4, size=3)

In [ ]:
img = helper.selection['run']['3732']['array']

plt.imshow(img)

In [ ]:
helper.get_crops(helper.selection['run_root_id'])

In [ ]:
crops = helper.selection['run']['3732']['crops']

In [ ]:
import numpy as np
import cv2

i = 2
buffer = 0

crop_bgr = cv2.cvtColor(crops[i], cv2.COLOR_RGB2BGR)

#set the lower and upper bounds for the green hue
lower = np.array([151-buffer,255-buffer,254-buffer])
upper = np.array([159+buffer,255,255])

#create a mask for green colour using inRange function
mask = cv2.inRange(crop_bgr, lower, upper)

#perform bitwise and on the original image arrays using the mask
res = cv2.bitwise_and(crops[i], crops[i], mask=mask)

plt.imshow(res)

In [ ]:
plt.imshow(crops[2])